In [2]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
from collections import deque
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

from importlib import reload
from utilities import *
from agent import DQNAgent

2024-04-17 01:44:42.155783: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
stocks = ['AAPL', 'GOOGL', 'AMZN', 'ADBE']

In [4]:
episode_count = 20
true, diff = allStates(1000)
split = int(0.8*len(true[0]))

true_train, true_test = true[:split], true[:split]
diff_train, diff_test = diff[:split], diff[:split]

l = len(diff_train) - 1

Model found for  AAPL
Model found for  GOOGL
Model found for  AMZN
Model found for  ADBE
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [5]:
p_agent = DQNAgent(4)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4, 64)          │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4, 3)           │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,307 (9.01 KB)

 Trainable params: 2,307 (9.01 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
for e in range(1,episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getstate(diff_train, 0)
    total_profit = 0
    p_agent.inventory = OrderedDict()
    
    for stock in stocks:
        p_agent.inventory[stock] = []

    for t in range(l):
        actions = p_agent.epsilon_greedy(state)
        # sit
        next_state = getstate(diff_train, t+1)
        reward = np.zeros((4,1))
        
        for i in range(len(actions)):
            action = actions[i]
            stock = stocks[i]
            
            if action == 1: # buy
                p_agent.inventory[stock].append(true_train[i,t])
                
            elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
                bought_price = p_agent.inventory[stock].pop(0)
                reward[i] = np.max(true_train[i,t] - bought_price, 0)
                # print(reward)
                total_profit += true_train[i,t] - bought_price

        done = True if t == l - 1 else False
        p_agent.store_transition(state, actions, reward, next_state, done)
        state = next_state

        if len(p_agent.replay_buffer) > 32:
            p_agent.exp_replay(32)

Episode 1/20
Episode 2/20
Episode 3/20
Episode 4/20
Episode 5/20
Episode 6/20
Episode 7/20
Episode 8/20
Episode 9/20
Episode 10/20
Episode 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━

In [8]:
state = getstate(diff_test, 0)
total_profit = 0
p_agent.inventory = OrderedDict()

for stock in stocks:
    p_agent.inventory[stock] = []

for t in range(l):
    actions = p_agent.epsilon_greedy(state)
    # sit
    next_state = getstate(diff_test, t+1)
    reward = np.zeros((4,1))
    
    for i in range(len(actions)):
        action = actions[i]
        stock = stocks[i]
        
        if action == 1: # buy
            p_agent.inventory[stock].append(true_test[i,t])
            print("Stock: ", stock, "Action: Buy: $", true_test[i,t])
            
        elif action == 2 and len(p_agent.inventory[stock]) > 0: # sell
            bought_price = p_agent.inventory[stock].pop(0)
            reward[i] = np.max(true_test[i,t] - bought_price, 0)
            # print(reward)
            total_profit += true_test[i,t] - bought_price
            print("Stock: ",stock,"Action: Sell: $", (true_test[i,t]), " | Profit: $",(true_test[i,t] - bought_price))

    done = True if t == l - 1 else False
    p_agent.store_transition(state, actions, reward, next_state, done)
    state = next_state
    if done:
            print("-----------------------------------")
            print(" Total Profit: " ,total_profit)
            print("-----------------------------------")

Stock:  GOOGL Action: Buy: $ 75.837502
Stock:  AMZN Action: Buy: $ 149.327499
Stock:  AAPL Action: Buy: $ 90.571907
Stock:  AMZN Action: Buy: $ 150.445496
Stock:  AAPL Action: Sell: $ 92.718452  | Profit: $ 2.1465450000000033
-----------------------------------
 Total Profit:  2.1465450000000033
-----------------------------------
